In [ ]:
import mne, os
from signal_processing.pre_process import *
from basic.arrange_data import read_files, create_results_folders
import numpy as np
from cmath import nan
import matplotlib.pyplot as plt
import pandas as pd

mne.set_log_level('error')

In [ ]:
exp_folder = r"Auditory Oddball/Baseline" # in case you want to just type the dir here
#exp_folder = input('Experiment folder (e.g., Eyes Open\Baseline):')

raw_folder = r"Data/Raw/OKTOS/"
clean_folder = r"Data/Clean/OKTOS"
dir_inprogress = os.path.join(raw_folder,exp_folder)
export_dir = os.path.join(clean_folder,exp_folder)

results_foldername = r"Results/OKTOS"
exp_condition = r"AO_T01"

file_dirs, subject_names = read_files(dir_inprogress,".bdf")

In [ ]:
def find_erp_peak(evoked,erp,time_window=[0,0.8],mode='pos',smallunits=True):
    if smallunits == True:
        time_coef = 1e3 # ms
        amplitude_coef= 1e6 # uV
    else:
        time_coef = 1 # s
        amplitude_coef = 1 # V
    
    try:
        _, erp[0], erp[1] = evoked.get_peak(ch_type='eeg',tmin=time_window[0],tmax=time_window[1],mode=mode,return_amplitude=True)
        erp[0] = erp[0] * time_coef
        erp[1] = erp[1] * amplitude_coef
    except:
        erp[0],erp[1] = nan,nan

    return erp


In [ ]:

dir_inprogress = os.path.join(clean_folder,exp_folder)
file_dirs, subject_names = read_files(dir_inprogress,"_clean-epo.fif")

# Predefinables
event_list = ['target after 1 standard','target after 3 standards','target after 5 standards',
            'target after 7 standards','target after 9 standards','target after 11 standards']
channel_picks = ['Cz'] ### This is the channel we are looking at, Jim told to look Fz, Cz and Pz
gfp = False
smallunits = True

# Time periods to look for ERPs
erp_time_n1 = [0.05, 0.15]
erp_time_n2 = [0.2, 0.35]
erp_time_p2 = [0.15, 0.28]
erp_time_p3 = [0.3, 0.5]
time_windows_log = str(('N1:',erp_time_n1,'N2:',erp_time_n2,'P2:',erp_time_p2,'P3:',erp_time_p3))

# Unit conversion
if smallunits == True:
    time_unit='ms'
    time_coef = 1e3
    amplitude_unit='uV'
    amplitude_coef= 1e6
else:
    time_unit='s'
    time_coef = 1
    amplitude_unit='V'
    amplitude_coef = 1

# If more than one channel is looked at, then colorcode the plot
if len(channel_picks) != 1:
    spatial_colors = True
else:
    spatial_colors = False

# Pre-create the variables for evoked objects and arrays to include ERP latencies and amplitudes
evoked_tar_grand = [None]*len(file_dirs)
evoked_tar = [None]*len(file_dirs)
for i in range(len(file_dirs)):
    evoked_tar[i] = [None]*len(event_list)
n1 = np.zeros(shape=(len(file_dirs),len(event_list),2)) # subject, event type, [latency, amplitude]
n2 = np.zeros(shape=(len(file_dirs),len(event_list),2))
p2 = np.zeros(shape=(len(file_dirs),len(event_list),2))
p3 = np.zeros(shape=(len(file_dirs),len(event_list),2))
n1_grand = np.zeros(shape=(len(file_dirs),2))
n2_grand = np.zeros(shape=(len(file_dirs),2))
p2_grand = np.zeros(shape=(len(file_dirs),2))
p3_grand = np.zeros(shape=(len(file_dirs),2))

# Go through all the files (subjects) in the folder
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress,subject_names[i]),verbose=False)
    
    ### Find ERPs for the grand average evokeds
    evoked_tar_grand[i] = epochs[event_list].average(picks=channel_picks)

    # N100 (fronto-central) -> 65-150ms
    n1_grand[i] = find_erp_peak(evoked_tar_grand[i],n1_grand[i],time_window=erp_time_n1,mode='neg',smallunits=True)

    # N200 (anterior) -> 200-350ms
    n2_grand[i] = find_erp_peak(evoked_tar_grand[i],n2_grand[i],time_window=erp_time_n2,mode='neg',smallunits=True)
    
    # P200 (centro-frontal, parieto-occipital) -> 150-280ms
    p2_grand[i] = find_erp_peak(evoked_tar_grand[i],p2_grand[i],time_window=erp_time_p2,mode='pos',smallunits=True)
    
    # P300 (parietal) -> 300-500ms whereas early window is 300-400ms and late is 380-440ms
    p3_grand[i] = find_erp_peak(evoked_tar_grand[i],p3_grand[i],time_window=erp_time_p3,mode='pos',smallunits=True)

    ### Find ERPs for the individual events
    for e in range(len(event_list)):
        evoked_tar[i][e] = epochs[event_list[e]].average(picks=channel_picks)

        # N100 (fronto-central) -> 65-150ms
        n1[i][e] = find_erp_peak(evoked_tar[i][e],n1[i][e],time_window=erp_time_n1,mode='neg',smallunits=True)
        
        # N200 (anterior) -> 200-350ms
        n2[i][e] = find_erp_peak(evoked_tar[i][e],n2[i][e],time_window=erp_time_n2,mode='neg',smallunits=True)
        
        # P200 (centro-frontal, parieto-occipital) -> 150-280ms
        p2[i][e] = find_erp_peak(evoked_tar[i][e],p2[i][e],time_window=erp_time_p2,mode='pos',smallunits=True)
        
        # P300 (parietal) -> 300-500ms whereas early window is 300-400ms and late is 380-440ms
        p3[i][e] = find_erp_peak(evoked_tar[i][e],p3[i][e],time_window=erp_time_p3,mode='pos',smallunits=True)
        
    # Plot the averaged evoked objects with peaks
    fig, axs = plt.subplots(nrows=len(event_list)+1, ncols=1,figsize=(8, 20), layout='tight')
    plt.suptitle('Evoked EEG with marked N1, N2, P2, P3 ({})'.format(subject_names[i]),y=1.01)
    for ix, ax in enumerate(axs):
        # Grand average of all stimulus
        if ix == 0:
            evoked_tar_grand[i].plot(axes=ax, show=False, time_unit='ms', titles='Grand average of all stimulus')
            ax.plot(n1_grand[i][0], n1_grand[i][1], marker='*', color='b')
            ax.axvspan(xmin=erp_time_n1[0]*time_coef,xmax=erp_time_n1[1]*time_coef, facecolor='b', alpha=0.1)
            ax.plot(n2_grand[i][0], n2_grand[i][1], marker='*', color='b')
            ax.axvspan(xmin=erp_time_n2[0]*time_coef,xmax=erp_time_n2[1]*time_coef, facecolor='c', alpha=0.1)
            ax.plot(p2_grand[i][0], p2_grand[i][1], marker='*', color='r')
            ax.axvspan(xmin=erp_time_p2[0]*time_coef,xmax=erp_time_p2[1]*time_coef, facecolor='r', alpha=0.1)
            ax.plot(p3_grand[i][0], p3_grand[i][1], marker='*', color='r')
            ax.axvspan(xmin=erp_time_p3[0]*time_coef,xmax=erp_time_p3[1]*time_coef, facecolor='g', alpha=0.1)
        # Separate stimulus averages
        else:
            title = '{}'.format(event_list[ix-1])
            evoked_tar[i][ix-1].plot(axes=ax, time_unit=time_unit, show=False, titles=title)
            ax.plot(n1[i][ix-1][0], n1[i][ix-1][1], marker='*', color='b')
            ax.axvspan(xmin=erp_time_n1[0]*time_coef,xmax=erp_time_n1[1]*time_coef, facecolor='b', alpha=0.1)
            ax.plot(n2[i][ix-1][0], n2[i][ix-1][1], marker='*', color='b')
            ax.axvspan(xmin=erp_time_n2[0]*time_coef,xmax=erp_time_n2[1]*time_coef, facecolor='c', alpha=0.1)
            ax.plot(p2[i][ix-1][0], p2[i][ix-1][1], marker='*', color='r')
            ax.axvspan(xmin=erp_time_p2[0]*time_coef,xmax=erp_time_p2[1]*time_coef, facecolor='r', alpha=0.1)
            ax.plot(p3[i][ix-1][0], p3[i][ix-1][1], marker='*', color='r')
            ax.axvspan(xmin=erp_time_p3[0]*time_coef,xmax=erp_time_p3[1]*time_coef, facecolor='g', alpha=0.1)


In [ ]:
df_erps = pd.DataFrame()
df_erps_grand = pd.DataFrame()
for i in range(len(file_dirs)):
    # Create a dataframe for individual stimulus conditions
    df_erps_temp = pd.DataFrame(np.hstack((n1[i], n2[i], p2[i], p3[i])), columns = ['N1 latency','N1 amplitude',
                                                                                    'N2 latency','N2 amplitude',
                                                                                    'P2 latency','P2 amplitude',
                                                                                    'P3 latency','P3 amplitude'])
    df_erps_temp['Stimulus'] = event_list
    df_erps_temp['Subject'] = subject_names[i]
    df_erps_temp['ERP detect-windows'] = time_windows_log
    df_erps = pd.concat([df_erps, df_erps_temp.set_index('Subject')])
    
    # Create a dataframe for grand average stimulus
    df_erps_grand_temp = pd.DataFrame(np.concatenate((n1_grand[i], n2_grand[i], p2_grand[i], p3_grand[i]))).T
    df_erps_grand_temp['Subject'] = subject_names[i]
    df_erps_grand_temp['ERP detect-windows'] = time_windows_log
    df_erps_grand_temp.columns = ['N1 latency','N1 amplitude','N2 latency','N2 amplitude',
                                'P2 latency','P2 amplitude','P3 latency','P3 amplitude',
                                'Subject','ERP detect-windows']
    df_erps_grand = pd.concat([df_erps_grand, df_erps_grand_temp.set_index('Subject')])
display(df_erps_grand)
display(df_erps)

Finding peaks for N1, N2, P2, P3 ERP components and plotting them to see if the peaks are in a 'logical' place; in case some of the peaks are not detected or are in a place which is not actually 'a peak', the peaks should be found manually. I would propose that to run all the participants at first and then go over visually the plots and write down participants which had a mis-identified peak (and which ERP it was) and afterwards run these participants again with wider time windows for finding the peaks.

Manual peak detection for averaged signals which are not good

In [ ]:
subject_name_manual = 'OKTOS_0002_00A_AO'

# NEW time periods to look for ERPs
erp_time_n1_manual = [0.05, 0.15]
erp_time_n2_manual = [0.2, 0.35]
erp_time_p2_manual = [0.1, 0.2]
erp_time_p3_manual = [0.27, 0.5]
time_windows_log_manual = str(('N1:',erp_time_n1_manual,'N2:',erp_time_n2_manual,'P2:',
                                erp_time_p2_manual,'P3:',erp_time_p3_manual))

# read the epoch, find the peaks the same way and plot
n1_grand_manual,n2_grand_manual,p2_grand_manual,p3_grand_manual = [0]*2,[0]*2,[0]*2,[0]*2
n1_manual = np.zeros(shape=(len(event_list),2))
n2_manual = np.zeros(shape=(len(event_list),2))
p2_manual = np.zeros(shape=(len(event_list),2))
p3_manual = np.zeros(shape=(len(event_list),2))
evoked_tar_manual = [None]*len(event_list)

# Read the clean data from the disk
epochs = mne.read_epochs(fname='{}/{}_clean-epo.fif'.format(dir_inprogress,subject_name_manual),verbose=False)

### Find ERPs for the grand average evokeds
evoked_tar_grand_manual = epochs[event_list].average(picks=channel_picks)

# N100 (fronto-central) -> 65-150ms
n1_grand_manual = find_erp_peak(evoked_tar_grand_manual,n1_grand_manual,time_window=erp_time_n1_manual,mode='neg',smallunits=True)

# N200 (anterior) -> 200-350ms
n2_grand_manual = find_erp_peak(evoked_tar_grand_manual,n2_grand_manual,time_window=erp_time_n2_manual,mode='neg',smallunits=True)

# P200 (centro-frontal, parieto-occipital) -> 150-280ms
p2_grand_manual = find_erp_peak(evoked_tar_grand_manual,p2_grand_manual,time_window=erp_time_p2_manual,mode='pos',smallunits=True)

# P300 (parietal) -> 300-500ms whereas early window is 300-400ms and late is 380-440ms
p3_grand_manual = find_erp_peak(evoked_tar_grand_manual,p3_grand_manual,time_window=erp_time_p3_manual,mode='pos',smallunits=True)

### Find ERPs for the individual events
for e in range(len(event_list)):
    evoked_tar_manual[e] = epochs[event_list[e]].average(picks=channel_picks)

    # N100 (fronto-central) -> 65-150ms
    n1_manual[e] = find_erp_peak(evoked_tar_manual[e],n1_manual[e],time_window=erp_time_n1_manual,mode='neg',smallunits=True)
    
    # N200 (anterior) -> 200-350ms
    n2_manual[e] = find_erp_peak(evoked_tar_manual[e],n2_manual[e],time_window=erp_time_n2_manual,mode='neg',smallunits=True)
    
    # P200 (centro-frontal, parieto-occipital) -> 150-280ms
    p2_manual[e] = find_erp_peak(evoked_tar_manual[e],p2_manual[e],time_window=erp_time_p2_manual,mode='pos',smallunits=True)
    
    # P300 (parietal) -> 300-500ms whereas early window is 300-400ms and late is 380-440ms
    p3_manual[e] = find_erp_peak(evoked_tar_manual[e],p3_manual[e],time_window=erp_time_p3_manual,mode='pos',smallunits=True)

# Plot the averaged evoked objects with peaks
#%matplotlib qt
fig, axs = plt.subplots(nrows=len(event_list)+1, ncols=1,figsize=(8, 20), layout='tight')
plt.suptitle('Evoked EEG with marked N1, N2, P2, P3 ({})'.format(subject_name_manual),y=1.01)
for ix, ax in enumerate(axs):
    # Grand average of all stimulus
    if ix == 0:
        evoked_tar_grand_manual.plot(axes=ax, show=False, time_unit='ms', titles='Grand average of all stimulus')
        ax.plot(n1_grand_manual[0], n1_grand_manual[1], marker='*', color='b')
        ax.axvspan(xmin=erp_time_n1_manual[0]*time_coef,xmax=erp_time_n1_manual[1]*time_coef, facecolor='b', alpha=0.1)
        ax.plot(n2_grand_manual[0], n2_grand_manual[1], marker='*', color='b')
        ax.axvspan(xmin=erp_time_n2_manual[0]*time_coef,xmax=erp_time_n2_manual[1]*time_coef, facecolor='c', alpha=0.1)
        ax.plot(p2_grand_manual[0], p2_grand_manual[1], marker='*', color='r')
        ax.axvspan(xmin=erp_time_p2_manual[0]*time_coef,xmax=erp_time_p2_manual[1]*time_coef, facecolor='r', alpha=0.1)
        ax.plot(p3_grand_manual[0], p3_grand_manual[1], marker='*', color='r')
        ax.axvspan(xmin=erp_time_p3_manual[0]*time_coef,xmax=erp_time_p3_manual[1]*time_coef, facecolor='g', alpha=0.1)
    # Separate stimulus averages
    else:
        title = '{}'.format(event_list[ix-1])
        evoked_tar_manual[ix-1].plot(axes=ax, time_unit='ms', show=False, titles=title)
        ax.plot(n1_manual[ix-1][0], n1_manual[ix-1][1], marker='*', color='b')
        ax.axvspan(xmin=erp_time_n1_manual[0]*time_coef,xmax=erp_time_n1_manual[1]*time_coef, facecolor='b', alpha=0.1)
        ax.plot(n2_manual[ix-1][0], n2_manual[ix-1][1], marker='*', color='b')
        ax.axvspan(xmin=erp_time_n2_manual[0]*time_coef,xmax=erp_time_n2_manual[1]*time_coef, facecolor='c', alpha=0.1)
        ax.plot(p2_manual[ix-1][0], p2_manual[ix-1][1], marker='*', color='r')
        ax.axvspan(xmin=erp_time_p2_manual[0]*time_coef,xmax=erp_time_p2_manual[1]*time_coef, facecolor='r', alpha=0.1)
        ax.plot(p3_manual[ix-1][0], p3_manual[ix-1][1], marker='*', color='r')
        ax.axvspan(xmin=erp_time_p3_manual[0]*time_coef,xmax=erp_time_p3_manual[1]*time_coef, facecolor='g', alpha=0.1)

#n1_grand_manual = [nan,nan]
#n1_manual[4] = [nan,nan]

df_erps_grand.loc[subject_name_manual,['N1 latency','N1 amplitude']] = n1_grand_manual
df_erps_grand.loc[subject_name_manual,['N2 latency','N2 amplitude']] = n2_grand_manual
df_erps_grand.loc[subject_name_manual,['P2 latency','P2 amplitude']] = p2_grand_manual
df_erps_grand.loc[subject_name_manual,['P3 latency','P3 amplitude']] = p3_grand_manual
df_erps_grand.loc[subject_name_manual,['ERP detect-windows']] = time_windows_log

df_erps.loc[subject_name_manual,['N1 latency','N1 amplitude']] = n1_manual
df_erps.loc[subject_name_manual,['N2 latency','N2 amplitude']] = n2_manual
df_erps.loc[subject_name_manual,['P2 latency','P2 amplitude']] = p2_manual
df_erps.loc[subject_name_manual,['P3 latency','P3 amplitude']] = p3_manual
df_erps.loc[subject_name_manual,['ERP detect-windows']] = time_windows_log

Once you are satisified with the new peak locations from the plot and/or have manually set some peaks from the block above, the block below will add all the new peak latencies and amplitudes to the main dataframes.

In [ ]:
# Pre-create results folders for spectral analysis data
create_results_folders(exp_folder=exp_folder,results_foldername=results_foldername)

df_erps_grand.to_excel(r"{}/{}_grandaverage_erps.xlsx".format(results_foldername,exp_condition))
df_erps.to_excel(r"{}/{}_conditionaverages_erps.xlsx".format(results_foldername,exp_condition))